<a href="https://colab.research.google.com/github/sparsh-tekriwal/Image-Recognition-Dogs-vs-Cats/blob/master/Image%20Classification%20using%20PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
REBUILD_DATA = True

In [ ]:
class DogsVSCats():
  IMG_SIZE = 50
  CATS = "PetImages/Cat"
  DOGS = "PetImages/Dog"
  LABELS = {CATS: 0, DOGS: 1}
  training_data = []
  catcount = 0
  dogcount = 0

  def make_training_data(self):
    # Iterating through Cats and Dogs directories
    for label in self.LABELS:
      print(label)
      # Iterating through contents of each directory along with a progress bar
      for f in tqdm(os.listdir(label))
        try:

          # joining file name to path
          path = os.path.join(label, f)
          #reading image file and converting to grayscale as my hypothesis is color doesn't matter while differenciating between cats and dogs. And we gotta simplify for efficient NN computation
          img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
          # I have defined this parmater for the class
          img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
          # converting labels to a one hot format
          self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

          # Counting number of data points in each class
          if label == self.CATS:
            self.catcount +=1
          elif label == self.DOGS:
            self.dogcount +=1
        
        except Exception as e:
          print("We have an issue with this dog/cat image: " + e)
    # Shuffling the ingested data so that they are evenly distributed
    # No reassignment requiremnt for this numpy function
    np.random.shuffle(self.training_data)
    # Saving data so that I don't need to import it do this computation again
    np.save("training_data.npy", self.training_data)
    print("Cats: ", self.catcount)
    print("Dogs: ", self.dogcount)

  if REBUILD_DATA:
    dogvcats = DogsVSCats()
    dogvcats.make_training_data()
    





In [4]:
training_data = np.load("training_data.npy")


x
y
